# MDPs and Q-learning On "Ice" (60 points possible)

In this assignment, we’ll revisit Markov Decision Processes while also trying out Q-Learning, the reinforcement learning approach that associates utilities with attempting actions in states.
The problem that we’re attempting to solve is the following:

1.  There is a grid of spaces in a rectangle.  Each space can contain a pit (negative reward), gold (positive reward), or nothing.
2.  The rectangle is effectively surrounded by walls, so anything that would move you outside the rectangle, instead moves you to the edge of the rectangle.
3.  The floor is icy.  Any attempt to move in a cardinal direction results in moving a somewhat random number of spaces in that direction.  The exact probabilities of moving each number of spaces are given in the problem description.  (If you slide too far, see rule #2.)
4.  Landing on a pit or gold effectively “ends the run,” for both a Q learner and an agent later trying out the policy.  It’s game over.  (To simulate this during Q learning, set all Q values for the space to equal its reward, then start over from a random space.)  Note that it's still possible to slide past a pit or gold - this doesn't end the run.

A sample input looks like this:


In [1]:
sampleMDP = """0.7 0.2 0.1
- - P - -
- - G P -
- - P - -
- - - - -"""


The first line says that the probabilities of moving one, two, or three spaces in the direction of movement are 0.7, 0.2, and 0.1.   The rest is a map of the environment, where a dash is an empty space, P is a pit, and G is gold.

Your job is to finish the code below for mdp_solve() and q_solve().  These take a problem description like the one pictured above, and return a policy giving the recommended action to take in each empty square (U=up, R=right, D=down, L=left).

**1, 19 points)**  mdp_solve() should use value iteration and the Bellman equation.  ITERATIONS will refer to the number of complete passes you perform over all states.  You can initialize the utilities to the rewards of each state.  Don’t update the rewards spaces from their initial rewards; since they end the trial, they have no future utility.  Don't update utilities in-place as you iterate through them, but create a fresh array of utilities with each pass, in order to avoid biasing moves in the directions that have already been updated.

**2, 26 points)**  q_solve() will run ITERATIONS trials in which a learner starts in a random empty square and moves until it hits a pit or gold, in which case, the trial is over.  (If it was randomly dropped into gold or a pit, the trial is immediately over.)  The learner moves by deciding randomly whether to choose a random direction (with probability EXPLORE_PROB) or move according to the best Q-value of its current square (otherwise).  Simulate the results of the move on slippery ice to determine where the learner ended up - then apply the Q-learning equation given in lecture and the textbook.  (There are multiple Q-learning variants out there, so try to use the equations and practices described in lecture instead of using other sources, to avoid confusion.)

The fact that a trial ends immediately on finding gold or a pit means that we want to handle those spaces in a special way.  Normally Q values are updated on moving to the next state, but we won’t see any next state in these cases.  So, to handle this, when the agent discovers one of these rewards, set all the Q values for that space to the associated reward before quitting the trial.  So, for example, if gold is worth 100 and it’s discovered in square x, Q(x,UP) = 100, Q(x,RIGHT) = 100, Q(x, DOWN) = 100, and Q(x, LEFT) = 100.  There’s no need to apply the rest of the Q update equation when the trial is ending, because that’s all about future rewards, and there’s no future when the trial is ending.  But now the spaces that can reach that space will evaluate themselves appropriately.  (Before being "discovered," the square should have no utility.)

You should use the GOLD_REWARD, PIT_REWARD, LEARNING_RATE, and DISCOUNT_FACTOR constants at the top of the code box below.

Q-learning involves a lot of randomness and some arbitrary decisions when breaking ties, so two implementations can both be correct but recommend slightly different policies in the end, even if they have the same starting random seed.  While we provide some helpful premade maps below, your main guide for debugging will be common sense in deciding whether the policy created by your agent makes sense -- ie, agents following the policy will get gold without taking unnecessary risks.

In [11]:
""" "MDPs on Ice - Assignment 5"""

import random
import copy
import numpy as np

GOLD_REWARD = 250.0
PIT_REWARD = -150.0
DISCOUNT_FACTOR = 0.8
EXPLORE_PROB = 0.2 # for Q-learning
LEARNING_RATE = 0.01
ITERATIONS = 100000
MAX_MOVES = 1000
ACTIONS = 4
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3
MOVES = ['U', 'R', 'D', 'L']

# Fixed random number generator seed for result reproducibility --
# don't use a random number generator besides this to match sol
random.seed(340)

class Problem:
    """Represents the physical space, transition probabilities, reward locations, and approach

    ...in short, the info in the problem string

    Attributes:
        move_probs (List[float]):  probabilities of going 1,2,3 spaces
        map (List[List(string)]]:  "-" (safe, empty space), "G" (gold), "P" (pit)

    String format consumed looks like
    0.7 0.2 0.1   [probability of going 1, 2, 3 spaces]
    - - - - - - P - - - -   [space-delimited map rows]
    - - G - - - - - P - -   [G is gold, P is pit]

    You can assume the maps are rectangular, although this isn't enforced
    by this constructor.
    """

    def __init__(self, probstring):
        """ Consume string formatted as above"""
        self.map = []
        for i, line in enumerate(probstring.splitlines()):
            if i == 0:
                self.move_probs = [float(s) for s in line.split()]
            else:
                self.map.append(line.split())

    def solve(self, iterations, use_q):
        """ Wrapper for MDP and Q solvers.

        Args:
            iterations (int):  Number of iterations (but these work differently for the two solvers)
            use_q (bool):  False means use MDP value iteration, true means use Q-learning
        Returns:
            A Policy, in either case (what to do in each square; see class below)
        """

        if use_q:
            return q_solve(self, iterations)
        return mdp_solve(self, iterations)

class Policy:
    """ Abstraction on the best action to perform in each state.

    This is a string list-of-lists map similar to the problem input, but a character gives the best
    action to take in each non-reward square (see MOVES constant at top of file).
    """

    def __init__(self, problem):
        """Args:

        problem (Problem):  The MDP problem this is a policy for
        """
        self.best_actions = copy.deepcopy(problem.map)

    def __str__(self):
        """Join the characters in the policy into one big space-separated, multline string"""
        return '\n{}\n'.format('\n'.join([' '.join(row) for row in self.best_actions]))

def roll_steps(move_probs, row, col, move, rows, cols):
    """Calculates the new coordinates that result from a move.

    Includes the "roll of the dice" for transition probabilities and checking arena boundaries.

    Helper for try_policy and q_solve - probably useful in your Q-learning implementation.

    Args:
        move_probs (List[float]):  Transition probabilities for the ice (from problem)
        row, col (int, int):  location of agent before moving
        move (string):  The direction of move as a MOVES character (not an int constant!)
        rows, cols (int, int):  number of rows and columns in the map

    Returns:
        new_row, new_col (int, int):  The new row and column after moving
    """
    displacement = 1
    total_prob = 0
    move_sample = random.random()
    for p, prob in enumerate(move_probs):
        total_prob += prob
        if move_sample <= total_prob:
            displacement = p+1
            break
    # Handle "slipping" into edge of map
    new_row = row
    new_col = col
    if not isinstance(move, str):
        print("Warning: roll_steps wants str for move, got a different type")
    if move == "U":
        new_row -= displacement
        if new_row < 0:
            new_row = 0
    elif move == "R":
        new_col += displacement
        if new_col >= cols:
            new_col = cols-1
    elif move == "D":
        new_row += displacement
        if new_row >= rows:
            new_row = rows-1
    elif move == "L":
        new_col -= displacement
        if new_col < 0:
            new_col = 0
    return new_row, new_col


def try_policy(policy, problem, iterations):
    """Returns average utility per move of the policy.

    Average utility is as measured by "iterations" random drops of an agent onto empty
    spaces, running until gold, pit, or time limit MAX_MOVES is reached.

    Doesn't necessarily play a role in your code, but you can try policies this
    way

    Args:
        policy (Policy):  the policy the agent is following
        problem (Problem):  the environment description
        iterations (int):  the number of random trials to run
    """
    total_utility = 0
    total_moves = 0
    for _ in range(iterations):
        # Resample until we have an empty starting square
        while True:
            row = random.randrange(0, len(problem.map))
            col = random.randrange(0, len(problem.map[0]))
            if problem.map[row][col] == "-":
                break
        for moves in range(MAX_MOVES):
            total_moves += 1
            policy_rec = policy.best_actions[row][col]
            # Take the move - roll to see how far we go, bump into map edges as necessary
            row, col = roll_steps(problem.move_probs, row, col, policy_rec, \
                                  len(problem.map), len(problem.map[0]))
            if problem.map[row][col] == "G":
                total_utility += GOLD_REWARD
                break
            if problem.map[row][col] == "P":
                total_utility += PIT_REWARD
                break
    return total_utility / total_moves

def mdp_solve(problem, iterations):
    """ Perform value iteration for the given number of iterations on the MDP problem.

    Here, the squares with rewards can be initialized to the reward values, since value iteration
    assumes complete knowledge of the environment and its rewards.

    Args:
        problem (Problem):  description of the environment
        iterations (int):  number of complete passes over the utilities
    Returns:
        a Policy (though you may design this to return utilities as a second return value)
    """
    # TODO calculate the policy

    # initialize blank map
    bellman_map = [[0]*len(problem.map[0]) for x in range(len(problem.map))]
    
    # initialize rewards map
    for row in range(len(problem.map)):
        index = -1
        for square in range(len(problem.map[row])):
            index += 1

            if problem.map[row][square] == 'P':
                bellman_map[row][index] = -150

            elif problem.map[row][square] == 'G':
                bellman_map[row][index] = 250

    # gloabl iteration loop
    for x in range(iterations):

      # completes appropriate bellman calculation
      def bellman(move, prev_util, distance):

          try:
              move_prob = problem.move_probs[distance]
          except IndexError:
              move_prob = 0

          if move == 'P':
              util = move_prob * (PIT_REWARD + DISCOUNT_FACTOR * prev_util)
          if move == 'G':
              util = move_prob * (GOLD_REWARD + DISCOUNT_FACTOR * prev_util)
          else:
              util = move_prob * (0 + DISCOUNT_FACTOR * prev_util)
        
          return util

      # neatly prints list of lists
      def prettyprint(map):
          for row in range(len(problem.map)):
              print('')
              for col in range(len(problem.map[0])):
                  print(str(map[row][col]) + (' ' * (6 - len(str(map[row][col])))), end='')

      # row index tracker
      rowNum = -1
      for row in problem.map:
          rowNum += 1

          for square in range(len(row)):

              if problem.map[rowNum][square] == "G":
                  bellman_map[rowNum][square] = 250
                  continue
              elif problem.map[rowNum][square] == "P":
                  bellman_map[rowNum][square] = -150
                  continue

              prev_util = bellman_map[rowNum][square]

              for move in range(ACTIONS):

                  if move == UP:
                      # upwards bellman logic
                      moveOutcomes = []
                      for i in range(rowNum-1, -1, -1):
                          moveOutcomes.append(problem.map[i][square])

                      if len(moveOutcomes) == 0:
                          upUtil = 0
                      else:
                          upUtil = 0
                          for i in range(len(moveOutcomes[:3])):
                              upUtil += bellman(moveOutcomes[i], prev_util, i)

                  elif move == RIGHT:
                      # right bellman logic
                      moveOutcomes = problem.map[rowNum][square+1:]
                      if len(moveOutcomes) == 0:
                          rightUtil = 0
                      else:
                          rightUtil = 0
                          for i in range(len(moveOutcomes[:3])):
                              rightUtil += bellman(moveOutcomes[i], prev_util, i)

                  elif move == DOWN:
                      # down bellman logic
                      moveOutcomes = []
                      for i in range(3-rowNum):
                          moveOutcomes.append(problem.map[rowNum+i+1][square])

                      if len(moveOutcomes) == 0:
                          downUtil = 0
                      else:
                          downUtil = 0
                          for i in range(len(moveOutcomes[:3])):
                              downUtil += bellman(moveOutcomes[i], prev_util, i)

                  elif move == LEFT:
                      # left bellman logic
                      if (square) < 0:
                          leftUtil = 0
                      else:
                          moveOutcomes = problem.map[rowNum][:square]

                          if len(moveOutcomes) == 0:
                              leftUtil = 0
                          else:
                              leftUtil = 0
                              if len(moveOutcomes) > 3:
                                  moveOutcomes = moveOutcomes[-3:]

                              for i in range(len(moveOutcomes)-1, -1, -1):
                                  leftUtil += bellman(moveOutcomes[i], prev_util, 2-i)

              # catches overflow bug for big_test
              try:
                  bellman_map[rowNum][square] = int(max(upUtil, rightUtil, downUtil, leftUtil))
              except OverflowError:
                  bellman_map[rowNum][square] = 10000
                  
    # debug code; prints input and output in neat format

    if (x == iterations-1):
        for row in problem.map:
          print('')
          for square in row:
            print(square + ' ', end='')
        
        print('\n')

        prettyprint(bellman_map)

        print('\n\n')

    return bellman_map

def q_solve(problem, iterations):
    """q_solve:  Use Q-learning to find a good policy on an MDP problem.

    Each iteration corresponds to a random drop of the agent onto the map, followed by moving
    the agent until a reward is reached or MAX_MOVES moves have been made.  When an agent
    is sitting on a reward, update the utility of each move from the space to the reward value
    and end the iteration.  (For simplicity, the agent also does this if just dropped there.)
    The agent does not "know" reward locations in its Q-values before encountering the space
    and "discovering" the reward.

    Note that texts differ on when to pay attention to this reward - this code follows the
    convention of scoring rewards of the space you are moving *from*, plus discounted best q-value
    of where you landed.

    Assume epsilon-greedy exploration.  Leave reward letters as-is in the policy,
    to make it more readable.

    Args:
        problem (Problem):  The environment
        iterations (int):  The number of runs from random start to reward encounter
    Returns:
        A Policy for the map
    """
    # TODO

    def initializeRewardsTable(problem):
      mdpIteration = []
      for row in range(len(problem.map)):
          temp = []
          for square in problem.map[row]:
              if square == 'P':
                  temp.append(-150)
              elif square == 'G':
                  temp.append(250)
              else:
                  temp.append(0)
          mdpIteration.append(temp)
      return mdpIteration


    policy = Policy(problem)
    rewards = initializeRewardsTable(problem)
    rows = len(rewards)
    cols = len(rewards[0])
    actions = 4
    Qtable = [[[0,0,0,0] for c in range(cols)] for r in range(rows)]
    updateQtable = copy.deepcopy(Qtable)
    
    for i in range(iterations):
      x = random.randrange(0, len(problem.map))
      y = random.randrange(0, len(problem.map[0]))
      count = 0

      while count <= MAX_MOVES:
        if rewards[x][y] == GOLD_REWARD or rewards[x][y] == PIT_REWARD:
          updateQtable[x][y] = [rewards[x][y]] * actions 
          break
        else:
          if random.uniform(0, 1) <= EXPLORE_PROB:
            movenum = random.randrange(0,4)
            new_x, new_y = roll_steps(problem.move_probs, x, y, MOVES[movenum], rows, cols)
            newQ = new_q(rewards, Qtable, x, y, new_x, new_y, movenum)
            updateQtable[x][y][movenum] = newQ
            x = new_x
            y = new_y
          else:
            maxQ = max(Qtable[x][y])
            movenum = Qtable[x][y].index(maxQ)
            new_x, new_y = roll_steps(problem.move_probs, x, y, MOVES[movenum], rows, cols)
            newQ = new_q(rewards, Qtable, x, y, new_x, new_y, movenum)
            updateQtable[x][y][movenum] = newQ
            x = new_x
            y = new_y
          count += 1
          Qtable = copy.deepcopy(updateQtable)

    for r in range(rows):
      for c in range(cols):
        if rewards[r][c] == 0:
          policy.best_actions[r][c] = str(Qtable[r][c].index(max(Qtable[r][c])))

    return policy

def new_q(rewards, utilities, r, c, new_r, new_c, movenum):
    """ Q-learning function.  Returns the new Q-value for space (r,c).
    It's recommended you code and test this before doing the overall Q-learning.

    Should use the LEARNING_RATE and DISCOUNT_FACTOR.

    Args:
        rewards (List[List[float]]):  Reward amounts built into the problem map (indexed [r][c])
        utilities (List[List[List[float]]]):  The Q-values for each action from each space.
                                              (Indexed as [row][col][move])
        r, c (int, int):  Row and column of our location before move
        new_r, new_c (int, int):  Row and column of our location after move
        movenum (int):  Integer index into the Q-values, corresponding to constants UP etc
    Returns:
        float - the new Q-value for the space we moved from
    """
    # TODO
    new_q = rewards[new_r][new_c] + LEARNING_RATE * ((rewards[r][c] + DISCOUNT_FACTOR * utilities[r][c][movenum]) - max(utilities[new_r][new_c]))
    return new_q

In [3]:
deterministic_test = """1.0
- - P - -
- - G P -
- - P - -
- - - - -"""

In [4]:
# Notice that we counterintuitively are most likely to go 2 spaces here
very_slippy_test = """0.2 0.7 0.1
- - P - -
- - G P -
- - P - -
- - - - -"""

In [5]:
big_test = """0.6 0.3 0.1
- P - G - P - - G -
P G - P - - - P - -
P P - P P - P - P -
P - - P P - - - - P
- - - - - - - - P G"""

In [15]:
# MDP value iteration tests
print(Problem(deterministic_test).solve(ITERATIONS, False))


- - P - - 
- - G P - 
- - P - - 
- - - - - 


0     0     -150  0     0     
0     1246  250   -150  0     
0     0     -150  0     0     
0     0     0     0     0     


[[0, 0, -150, 0, 0], [0, 1246, 250, -150, 0], [0, 0, -150, 0, 0], [0, 0, 0, 0, 0]]


In [14]:
print(Problem(sampleMDP).solve(ITERATIONS, False))


- - P - - 
- - G P - 
- - P - - 
- - - - - 


0     0     -150  0     0     
245   871   250   -150  245   
0     0     -150  0     0     
0     0     245   0     0     


[[0, 0, -150, 0, 0], [245, 871, 250, -150, 245], [0, 0, -150, 0, 0], [0, 0, 245, 0, 0]]


In [13]:
print(Problem(very_slippy_test).solve(ITERATIONS, False))


- - P - - 
- - G P - 
- - P - - 
- - - - - 


0     0     -150  0     0     
871   245   250   -150  871   
0     0     -150  0     0     
0     0     871   0     0     


[[0, 0, -150, 0, 0], [871, 245, 250, -150, 871], [0, 0, -150, 0, 0], [0, 0, 871, 0, 0]]


In [12]:
print(Problem(big_test).solve(ITERATIONS, False))


- P - G - P - - G - 
P G - P - - - P - - 
P P - P P - P - P - 
P - - P P - - - - P 
- - - - - - - - P G 


121   -150  746   250   746   -150  371   533   250   746   
-150  250   109   -150  121   0     0     -150  287   0     
-150  -150  0     -150  -150  0     -150  0     -150  0     
-150  371   0     -150  -150  0     0     0     121   -150  
0     121   0     0     0     0     121   265   -150  250   


[[121, -150, 746, 250, 746, -150, 371, 533, 250, 746], [-150, 250, 109, -150, 121, 0, 0, -150, 287, 0], [-150, -150, 0, -150, -150, 0, -150, 0, -150, 0], [-150, 371, 0, -150, -150, 0, 0, 0, 121, -150], [0, 121, 0, 0, 0, 0, 121, 265, -150, 250]]


In [ ]:
# Q-learning tests
# Set seed every time for consistent executions;
# comment out to get different random runs
random.seed(340)
print(Problem(deterministic_test).solve(ITERATIONS, True))


0 0 P 0 0
0 1 G P 0
0 2 P 1 0
0 0 1 0 0



In [ ]:
random.seed(340)
print(Problem(sampleMDP).solve(ITERATIONS, True))


0 0 P 0 0
1 0 G P 0
2 2 P 1 1
2 2 1 1 0



In [ ]:
random.seed(340)
print(Problem(very_slippy_test).solve(ITERATIONS, True))


0 0 P 0 0
0 0 G P 0
2 2 P 1 1
2 0 1 1 1



In [ ]:
random.seed(340)
print(Problem(big_test).solve(ITERATIONS, True))


3 P 2 G 3 P 3 1 G 0
P G 2 P 0 2 3 P 0 1
P P 2 P P 2 P 2 P 1
P 2 2 P P 2 2 3 3 P
2 3 1 2 2 3 3 3 P G



Once you're done, here are a few thought questions (15 points total):

**3, 3 points) In an earlier version of this code, students sometimes found that some bad luck with pits near the end of the Q-learning training could cause square (0,1) of sampleMDP to believe that going right was a bad idea.  What parameter would be best to adjust in order to mitigate the effect of bad luck near the end of a run?  In what direction should this parameter be adjusted?**

**The best parameter to adjust would be to adjust use_q to TRUE, it should be applied to go in the down direction.**



**4, 3 points) The value iteration MDP solver updates all squares an equal number of times.  The Q-learner does not.  Which squares might we expect the Q-learner to update the most?**

**We should expect the squares around the Golds and Pits to be updated the most compared to other squares.**

**5, 9 points) Suppose we change the state information so that, instead of knowing its coordinates on the map, the agent instead knows just the locations of all rewards in a 5x5 square with the agent at the square center.  Thus, at the start of every run, it may not know exactly where it is, but it knows what is in the vicinity.  It also does not know the transition model.**

**a, 3 points) We can't use value iteration here.  Why?**

**b, 3 points) How many state-action combinations are possible, assuming the contents of the agent's own square don't matter?  Is a lookup table of Q-values feasible if we allocate memory for each possible state-action combination?  (Let's define "feasible" as "able to be stored in a gig or less of memory," assuming 64-bit values.)**

**c, 3 points) What is the most likely method that we would use to generalize in Q-learning from seen state-action combinations to situations that haven't been encountered before?**


**a) We can't use value iteration since we have a known rewards locations on the map so we cannot start a random value function since our values are based on the rewards locations.**

**b) There would be 5^25 combination possibilities which would make the amount of tables needed to create this very challenging so your typical computer will msot likely run our of memory.**

**c) We would use some type of non-Sarsa method to generalie a Q-learning function from seen state-action combinations to situations that haven't been encountered before.**

**Remember to submit your code on Blackboard as both an .ipynb (File->Download->.ipynb) and a PDF (Print->Save as PDF).**